In [3]:
mot_table = {}

with open('mot_table.txt', 'r') as mot_file:
    mot_lines = mot_file.readlines()

for line in mot_lines:
    name, opcode, size, mtype = line.strip().split(',')
    mot_table[name] = (opcode, int(size), mtype)

mot_table

{'ADD': ('01', 2, 'IS'),
 'SUB': ('02', 2, 'IS'),
 'MOVR': ('03', 2, 'IS'),
 'MOVM': ('04', 2, 'IS'),
 'JMP': ('05', 2, 'IS'),
 'JNZ': ('06', 2, 'IS'),
 'INC': ('07', 2, 'IS'),
 'DEC': ('08', 2, 'IS'),
 'START': ('09', 0, 'AD'),
 'END': ('0A', 0, 'AD'),
 'LTORG': ('0B', 0, 'AD'),
 'DC': ('0C', 1, 'DL'),
 'DS': ('0D', 0, 'DL'),
 'STOP': ('0E', 0, 'AD'),
 'BC': ('0F', 0, 'AD'),
 'MOVE': ('10', 2, 'IS'),
 'ORG': ('11', 0, 'AD'),
 'MUL': ('12', 2, 'IS'),
 'READ': ('13', 0, 'AD')}

In [4]:
lc = 100
lines = []
words = []

with open('asm_code.txt', 'r') as asm_file:
    asm_lines = asm_file.readlines()

for line in asm_lines:
    lines.append(line.strip())
    word = line.strip().split()
    if not word:
        continue  # Skip empty lines
    words.append(word)

    instruction = word[0]
    operands = ' '.join(word[1:])

    if instruction == "START":
        lc = int(operands)
        continue

lc
lines
words

[['START', '400'],
 ['READ', 'A'],
 ['A', 'DS', '5'],
 ['MOVE', 'AREG', 'A'],
 ['ADD', 'AREG', '=5'],
 ['B', 'DC', '4'],
 ['READ', 'B'],
 ['MUL', 'BREG', '=4'],
 ['LTORG'],
 ['NEXT', 'SUB', 'AREG', 'BREG'],
 ['ORG', 'NEXT', '+4'],
 ['BC', 'A'],
 ['MUL', 'AREG', '=2'],
 ['ADD', 'AREG', 'BREG'],
 ['S1', 'MOVE', 'B', 'AREG'],
 ['LTORG'],
 ['ORG', 'S1', '+10'],
 ['MOVE', 'CREG', '=25'],
 ['ADD', 'CREG', '=12'],
 ['S3', 'MOVE', 'C', 'CREG'],
 ['LTORG'],
 ['ADD', 'CREG', '=1'],
 ['MUL', 'AREG', 'CREG'],
 ['SUB', 'AREG', '=2'],
 ['MOVE', 'AREG', 'C'],
 ['C', 'DS', '5'],
 ['BC', 'S3'],
 ['ORG', 'C', '+6'],
 ['STOP'],
 ['END']]

In [5]:
adjusted_code = []

for line in words:
    num_components = len(line)
    if num_components == 4:
        adjusted_code.append(line)
    elif num_components == 3:
        if (line[1] == 'DC' or line[1] == 'DS'):
            adjusted_code.append(line + [''])
        else:    
            adjusted_code.append([''] + line)
    elif num_components == 2:
        adjusted_code.append([''] + line + [''])
    elif num_components == 1:
        adjusted_code.append([''] + line + ['', ''] )

adjusted_code


[['', 'START', '400', ''],
 ['', 'READ', 'A', ''],
 ['A', 'DS', '5', ''],
 ['', 'MOVE', 'AREG', 'A'],
 ['', 'ADD', 'AREG', '=5'],
 ['B', 'DC', '4', ''],
 ['', 'READ', 'B', ''],
 ['', 'MUL', 'BREG', '=4'],
 ['', 'LTORG', '', ''],
 ['NEXT', 'SUB', 'AREG', 'BREG'],
 ['', 'ORG', 'NEXT', '+4'],
 ['', 'BC', 'A', ''],
 ['', 'MUL', 'AREG', '=2'],
 ['', 'ADD', 'AREG', 'BREG'],
 ['S1', 'MOVE', 'B', 'AREG'],
 ['', 'LTORG', '', ''],
 ['', 'ORG', 'S1', '+10'],
 ['', 'MOVE', 'CREG', '=25'],
 ['', 'ADD', 'CREG', '=12'],
 ['S3', 'MOVE', 'C', 'CREG'],
 ['', 'LTORG', '', ''],
 ['', 'ADD', 'CREG', '=1'],
 ['', 'MUL', 'AREG', 'CREG'],
 ['', 'SUB', 'AREG', '=2'],
 ['', 'MOVE', 'AREG', 'C'],
 ['C', 'DS', '5', ''],
 ['', 'BC', 'S3', ''],
 ['', 'ORG', 'C', '+6'],
 ['', 'STOP', '', ''],
 ['', 'END', '', '']]

In [6]:
mnemonics = []
symbols = []
pool_table = []
pool_count = 1
pool_index = 1
pool_count_index = []
literals = []

pool_count_index.append(pool_index)
for i in range(len(adjusted_code)):
    mnemonics.append(adjusted_code[i][1])
    if (adjusted_code[i][1] == 'LTORG'):
        pool_count_index.append(pool_index)
        pool_count = pool_count + 1
    if (adjusted_code[i][0] != ''):
        symbols.append(adjusted_code[i][0])
    if (adjusted_code[i][1].startswith("=")):
        pool_table.append([adjusted_code[i][1] , pool_count])
        pool_index = pool_index + 1
        literals.append(adjusted_code[i][3])
    if (adjusted_code[i][3].startswith("=")):
        pool_table.append([adjusted_code[i][3] , pool_count])
        pool_index = pool_index + 1
        literals.append(adjusted_code[i][3])

# mnemonics
# symbols
# literals
pool_table
# pool_count_index

[['=5', 1], ['=4', 1], ['=2', 2], ['=25', 3], ['=12', 3], ['=1', 4], ['=2', 4]]

In [7]:
mnemonics = list(set(mnemonics))
mnemonics

['START',
 'BC',
 'DS',
 'ADD',
 'LTORG',
 'STOP',
 'DC',
 'ORG',
 'END',
 'MOVE',
 'MUL',
 'SUB',
 'READ']

In [8]:
mot_table_used = {}

for mnemonic in mnemonics:
    if mnemonic in mot_table:
        mot_table_used[mnemonic] = mot_table[mnemonic]
    else:
        raise ValueError(f"Mnemonic '{mnemonic}' not found in mot_table. Execution stopped.")

mot_table_used

{'START': ('09', 0, 'AD'),
 'BC': ('0F', 0, 'AD'),
 'DS': ('0D', 0, 'DL'),
 'ADD': ('01', 2, 'IS'),
 'LTORG': ('0B', 0, 'AD'),
 'STOP': ('0E', 0, 'AD'),
 'DC': ('0C', 1, 'DL'),
 'ORG': ('11', 0, 'AD'),
 'END': ('0A', 0, 'AD'),
 'MOVE': ('10', 2, 'IS'),
 'MUL': ('12', 2, 'IS'),
 'SUB': ('02', 2, 'IS'),
 'READ': ('13', 0, 'AD')}

In [9]:
# size = []
# opcode = []
# mtype = []

def get_size(mnemonic):
    if mnemonic in mot_table_used:
        return mot_table_used[mnemonic][1]
    else:
        raise ValueError(f"Mnemonic '{mnemonic}' size not found in mot_table. Execution stopped.")

def get_opcode(mnemonic):
    if mnemonic in mot_table_used:
        return mot_table_used[mnemonic][0]
    else:
        raise ValueError(f"Mnemonic '{mnemonic}' opcode not found in mot_table. Execution stopped.")

def get_mtype(mnemonic):
    if mnemonic in mot_table_used:
        return mot_table_used[mnemonic][2]
    else:
        raise ValueError(f"Mnemonic '{mnemonic}' mtype not found in mot_table. Execution stopped.")


found = False
def get_pool_table_index(literal):
    for literall in pool_table:
        if literal[0] == literall[0]:
            # print(literal[1])
            return literal[1]
    if found:
        pass
    else:
        raise ValueError(f"Literal'{literal}' not found in pool_table. Execution stopped.")

# mnemonic = 'MOVM'
# get_size(mnemonic)
# get_opcode(mnemonic)
# get_mtype(mnemonic)

# for mnemonic in mnemonics:
#     size.append(get_size(mnemonic))
#     opcode.append(get_opcode(mnemonic))
#     mtype.append(get_mtype(mnemonic))

# size
# opcode
# mtype

# get_pool_table_index("='5'")

In [10]:
lc_start = lc
lc_end = lc
symbol_address = []
literal_address = []
pool_table_count = 1
# size_list = []

for i in range(len(adjusted_code)):
    mnemonic = adjusted_code[i][1]
    size = get_size(mnemonic)
    # size_list.append(size)
    if (adjusted_code[i][0] != ''):
        symbol_address.append(lc_end)

    if (adjusted_code[i][1] == 'LTORG'):
        for literal in pool_table:
            if (get_pool_table_index(literal) == pool_table_count):
                literal_address.append(lc_end)
                lc_end = lc_end + 1
        pool_table_count = pool_table_count + 1

    lc_end = lc_end + size

# print(lc_end)

for literal in pool_table:
    if (get_pool_table_index(literal) == pool_table_count):
        literal_address.append(lc_end)
        lc_end = lc_end + 1


# lc_start
# lc_end
# size_list
# symbol_address
literal_address
# pool_table_count


[407, 408, 417, 424, 425, 434, 435]

In [11]:
symbol_table = {}
count = 1

for symbol, address in zip(symbols, symbol_address): # In this code, the zip() function is used to iterate over both the symbols and symbol_address lists simultaneously
    symbol_table[symbol] = (address, count)
    count = count + 1

symbol_table

{'A': (400, 1),
 'B': (404, 2),
 'NEXT': (409, 3),
 'S1': (415, 4),
 'S3': (422, 5),
 'C': (434, 6)}

In [12]:
literal_table = []
count = 1

for literal, address in zip(literals, literal_address): # In this code, the zip() function is used to iterate over both the symbols and symbol_address lists simultaneously
    literal_table.append([literal ,address, count])
    count = count + 1

literal_table

[['=5', 407, 1],
 ['=4', 408, 2],
 ['=2', 417, 3],
 ['=25', 424, 4],
 ['=12', 425, 5],
 ['=1', 434, 6],
 ['=2', 435, 7]]

In [13]:
registers = []

for i in range(len(adjusted_code)):
    if not adjusted_code[i][2].isdigit() and adjusted_code[i][2] not in symbols and adjusted_code[i][2] != '':
        registers.append(adjusted_code[i][2])

registers

['AREG',
 'AREG',
 'BREG',
 'AREG',
 'AREG',
 'AREG',
 'CREG',
 'CREG',
 'CREG',
 'AREG',
 'AREG',
 'AREG']

In [14]:
registers = list(set(registers))
registers

['CREG', 'BREG', 'AREG']

In [15]:
register_table = {}
count = 1

for register in registers:
    register_table[register] = count
    count = count + 1

register_table

{'CREG': 1, 'BREG': 2, 'AREG': 3}

In [16]:
with open('output_tables.txt', 'w') as output_file:
    output_file.write("MOT Table:\n\n")
    output_file.write("Mnemonic\tOpcode\tSize\tMType\n")
    for mnemonic, (opcode, size, mtype) in mot_table_used.items():
        if (len(mnemonic) == 4):
            mnemonic = mnemonic + " "
        if (len(mnemonic) == 3):
            mnemonic = mnemonic + " " + " "
        if (len(mnemonic) == 2):
            mnemonic = mnemonic + " " + " " + " "
        output_file.write(f"{mnemonic}\t\t  {opcode}\t {size}\t\t {mtype}\n")
    
    output_file.write("\n\n\nSymbol Table:\n\n")
    output_file.write("Index\tSymbol\tAddress\n")
    for symbol, (address, count) in symbol_table.items():
        # if (len(symbol) == 4):
        #     symbol = symbol + " "
        # if (len(symbol) == 3):
        #     symbol = symbol + " " + " "
        if (len(symbol) == 2):
            symbol = symbol + " " + " " + " "
        if (len(symbol) == 1):
            symbol = symbol + " " + " " + " " + " "
        output_file.write(f"{count}\t\t{symbol}\t  {address}\n")

    output_file.write("\n\n\nPool Table:\n\n")
    output_file.write("Index\n")
    for count in pool_count_index:
        output_file.write(f"{count}\t\t \n")


    output_file.write("\n\n\nLiteral Table:\n\n")
    output_file.write("Index\tLiteral\t Address\n")
    for literal, address, count in literal_table:
        if (len(literal) == 2):
            literal = literal + " "
        output_file.write(f"{count}\t\t {literal}\t  {address}\n")

    output_file.write("\n\n\nRegister Table:\n\n")
    output_file.write("Index\tRegister\n")
    for register, index in register_table.items():
        output_file.write(f"{index}\t\t {register}\n")

In [17]:
def check_table(table, expected_values):
    for key, values in table.items():
        if isinstance(values, tuple) and len(values) != expected_values:
            raise ValueError(f"Table '{table}' has an incomplete entry for key '{key}'.")

# Define the expected number of values for each table
expected_values_mot = 3
expected_values_symbol = 2
expected_values_literal = 2
expected_values_register = 1

# Check each table
check_table(mot_table_used, expected_values_mot)
check_table(symbol_table, expected_values_symbol)
# check_table(literal_table, expected_values_literal)
check_table(register_table, expected_values_register)

# The code will only reach this point if all tables are complete
print("All tables are complete.")

All tables are complete.


In [18]:
def get_register_index(register):
    if register in register_table:
        return register_table[register]
    else:
        return None
    
def get_symbol_index(symbol):
    if symbol in symbol_table:
        return symbol_table[symbol][1]
    else:
        return None

found = False
count = 1 
def get_literal_index(literal):
    global count
    line = 0
    for literall in literal_table:
        line = line + 1
        if line == count:  
            if literal == literall[0]:
                count = count + 1
                return literall[2]
    if found:
        pass    
    else:
        return None

correct = False    
def check_literal(literal):
    for literall in literal_table:
        if literal == literall[0]:
            return True
    if correct:
        pass
    else:
        return False

In [19]:
intermediate_code = []

for line in adjusted_code:
    mnemonic = line[1]
    if mnemonic in mot_table_used:
        opcode, _, mtype = mot_table_used[mnemonic]
        intermediate_code.append([(mtype, opcode)])

    operand1 = line[2]
    if operand1.isdigit():
        intermediate_code[-1].append(('C', operand1))
    elif operand1 in register_table:
        intermediate_code[-1].append(('R', get_register_index(operand1)))
    elif operand1 in symbol_table:
        intermediate_code[-1].append(('S', get_symbol_index(operand1)))
    else:
        pass
    
    operand2 = line[3]
    if operand2.isdigit():
        intermediate_code[-1].append(('C', operand2))
    elif operand2 in register_table:
        intermediate_code[-1].append(('R', get_register_index(operand2)))
    elif operand2 in symbol_table:
        intermediate_code[-1].append(('S', get_symbol_index(operand2)))
    elif check_literal(operand2):
        intermediate_code[-1].append(('L', get_literal_index(operand2)))
    else:
        pass    

intermediate_code

[[('AD', '09'), ('C', '400')],
 [('AD', '13'), ('S', 1)],
 [('DL', '0D'), ('C', '5')],
 [('IS', '10'), ('R', 3), ('S', 1)],
 [('IS', '01'), ('R', 3), ('L', 1)],
 [('DL', '0C'), ('C', '4')],
 [('AD', '13'), ('S', 2)],
 [('IS', '12'), ('R', 2), ('L', 2)],
 [('AD', '0B')],
 [('IS', '02'), ('R', 3), ('R', 2)],
 [('AD', '11'), ('S', 3)],
 [('AD', '0F'), ('S', 1)],
 [('IS', '12'), ('R', 3), ('L', 3)],
 [('IS', '01'), ('R', 3), ('R', 2)],
 [('IS', '10'), ('S', 2), ('R', 3)],
 [('AD', '0B')],
 [('AD', '11'), ('S', 4)],
 [('IS', '10'), ('R', 1), ('L', 4)],
 [('IS', '01'), ('R', 1), ('L', 5)],
 [('IS', '10'), ('S', 6), ('R', 1)],
 [('AD', '0B')],
 [('IS', '01'), ('R', 1), ('L', 6)],
 [('IS', '12'), ('R', 3), ('R', 1)],
 [('IS', '02'), ('R', 3), ('L', 7)],
 [('IS', '10'), ('R', 3), ('S', 6)],
 [('DL', '0D'), ('C', '5')],
 [('AD', '0F'), ('S', 5)],
 [('AD', '11'), ('S', 6)],
 [('AD', '0E')],
 [('AD', '0A')]]

In [20]:
with open('intermediate_code.txt', 'w') as output_file:
    # output_file.write("Intermediate Code:\n\n")
    for line in intermediate_code:
        if len(line) == 3:
            (mtype, opcode), (operand1, value1), (operand2, value2) = line
            output_file.write(f"({mtype}, {opcode}) ({operand1}, {value1}) ({operand2}, {value2})\n")
        if len(line) == 2:
            (mtype, opcode), (operand1, value1) = line
            output_file.write(f"({mtype}, {opcode}) ({operand1}, {value1})\n")
        elif len(line) == 1:
            (mtype, opcode) = line[0]
            output_file.write(f"({mtype}, {opcode})\n")


In [21]:
# with open('used_mot_table.txt', 'w') as output_file:
#     for mnemonic, (opcode, size, mtype) in mot_table_used.items():
#         output_file.write(f"{mnemonic},{opcode},{size},{mtype}\n")

with open('symbol_table.txt', 'w') as output_file:
    for symbol, (address, count) in symbol_table.items():
        output_file.write(f"{count},{symbol},{address}\n")


with open('literal_table.txt', 'w') as output_file:
    for literal, address, count in literal_table:
        output_file.write(f"{count},{literal},{address}\n")

with open('register_table.txt', 'w') as output_file:
    for register, index in register_table.items():
        output_file.write(f"{index},{register}\n")